In [1]:
# Install once only - python official library from AWS
# !pip install boto3

### Define the Configuration

In [2]:
aws_accesskey = <Your Access Key>
aws_secretaccess = <Your secret access key>
myregion= <your region>

In [3]:
import boto3

In [4]:
Rekognition_client = boto3.client('rekognition',aws_access_key_id = aws_accesskey ,aws_secret_access_key = aws_secretaccess ,region_name = myregion )

In [5]:
from botocore.exceptions import ClientError

### Creating a collection

In [6]:
def Create_Collection(aws_access,aws_secret, aws_region,Collection_ID):
    print('Creating collection:' + Collection_ID)
    response = Rekognition_client.create_collection(CollectionId=Collection_ID)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')

In [7]:
Create_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection")

Creating collection:MyCollection
Collection ARN: aws:rekognition:ap-southeast-1:083587501290:collection/MyCollection
Status code: 200
Done...


In [8]:
Create_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection2")

Creating collection:MyCollection2
Collection ARN: aws:rekognition:ap-southeast-1:083587501290:collection/MyCollection2
Status code: 200
Done...


In [9]:
Create_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection3")

Creating collection:MyCollection3
Collection ARN: aws:rekognition:ap-southeast-1:083587501290:collection/MyCollection3
Status code: 200
Done...


In [10]:
Create_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection4")

Creating collection:MyCollection4
Collection ARN: aws:rekognition:ap-southeast-1:083587501290:collection/MyCollection4
Status code: 200
Done...


### Listing out all the collections

In [11]:
def List_Collection(aws_access,aws_secret, aws_region):
    print('Displaying collections: ')
    
    max_results=2
    
    response = Rekognition_client.list_collections(MaxResults=max_results)
    collection_count = 0
    done = False
    
    while done == False:
        collections = response['CollectionIds']
        
        for collection in collections:
            print(collection)
            collection_count+=1
        if 'NextToken' in response:
            nextToken = response['NextToken']
            response=Rekognition_client.list_collections(NextToken=nextToken,MaxResults=max_results)
            
        else:
            done = True
            
    return collection_count

In [12]:
List_Collection(aws_accesskey,aws_secretaccess,myregion)

Displaying collections: 
MyCollection
MyCollection2
MyCollection3
MyCollection4


4

### Deleting a Collection

In [13]:
def Delete_Collection(aws_access,aws_secret, aws_region,Collection_ID):
    print('Deleting collection: ' + Collection_ID)
    
    status_code = 0
    
    try:
        response = Rekognition_client.delete_collection(CollectionId=Collection_ID)
        status_code = response['StatusCode']
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print('The collection ' + Collection_ID + ' was not found ')
        else:
            print('Error other than Not Found occurred: ' + e.response['Error']['Message'])
        status_code = e.response['ResponseMetadata']['HTTPStatusCode']
    return (status_code)

In [14]:
Delete_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection3")

Deleting collection: MyCollection3


200

In [15]:
Delete_Collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection4")

Deleting collection: MyCollection4


200

### Adding Faces to the collection

In [16]:
def AddFaces_Collection(aws_access,aws_secret, aws_region,bucket_name,photo_name,Collection_ID):
    response = Rekognition_client.index_faces(CollectionId=Collection_ID,
                                  Image={'S3Object': {'Bucket': bucket_name, 'Name': photo_name}},
                                  ExternalImageId=photo_name,
                                  MaxFaces=15,
                                  QualityFilter="AUTO",
                                  DetectionAttributes=['ALL'])
    print('Results for ' + photo_name)
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
        print('  Face ID: ' + faceRecord['Face']['FaceId'])
        print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))
        
    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])

In [17]:
AddFaces_Collection(aws_accesskey,aws_secretaccess,myregion,<S3 bucket name>,'Celebrities_Group.jpg',"MyCollection")

Results for Celebrities_Group.jpg
Faces indexed:
  Face ID: ac5b2d79-a2d2-466c-aa66-eb7dc07b7828
  Location: {'Width': 0.1918514519929886, 'Height': 0.4871792495250702, 'Left': 0.5067951679229736, 'Top': 0.4071531891822815}
  Face ID: 634517bb-5347-4a7a-9232-fa3a108241ba
  Location: {'Width': 0.14550478756427765, 'Height': 0.3137165307998657, 'Left': 0.09954940527677536, 'Top': 0.12622591853141785}
  Face ID: 0a718a93-46fa-496d-a1db-107f27515ca3
  Location: {'Width': 0.13213908672332764, 'Height': 0.3438571095466614, 'Left': 0.3283844292163849, 'Top': 0.39797061681747437}
  Face ID: 61520c35-830a-4034-9a27-449928a0ce5d
  Location: {'Width': 0.1347799301147461, 'Height': 0.3243994414806366, 'Left': 0.7409043908119202, 'Top': 0.2618715167045593}
  Face ID: 61623f99-33f9-4fc2-8410-c0a45411d175
  Location: {'Width': 0.10224471241235733, 'Height': 0.3038819432258606, 'Left': 0.22581912577152252, 'Top': 0.3236635625362396}
  Face ID: 5a5821cc-00ea-4cfc-bd2b-13aad40500fa
  Location: {'Width':

11

### Describing a collection

In [18]:
def describe_collection(aws_access,aws_secret, aws_region,Collection_ID):
    try:
        response = Rekognition_client.describe_collection(CollectionId=Collection_ID)
        print("Collection Arn: " + response['CollectionARN']) # CollectionARN full form is collection Amazon Resource Name
        print("Face Count: " + str(response['FaceCount']))
        print("Face Model Version: " + response['FaceModelVersion'])
        print("Timestamp: " + str(response['CreationTimestamp']))
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print('The Collection ' + Collection_ID + ' was not found ')
        else:
            print('Error other than Not Found occurred: ' + e.response['Error']['Message'])
            print('Done...')

In [19]:
describe_collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection")

Collection Arn: arn:aws:rekognition:ap-southeast-1:083587501290:collection/MyCollection
Face Count: 11
Face Model Version: 7.0
Timestamp: 2024-03-08 17:44:26.420000+08:00


### Deleting Faces from a collection

In [20]:
def deleteFaces_collection(aws_access,aws_secret, aws_region,Collection_ID,faces):
    response = Rekognition_client.delete_faces(CollectionId=Collection_ID,FaceIds=faces)
    
    print(str(len(response['DeletedFaces'])) + ' faces deleted:')
    for faceId in response['DeletedFaces']:
        print(faceId)
    return len(response['DeletedFaces'])

    Collection_ID = 'collection-id'
    faces = []
    faces.append("xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx")

    faces_count = delete_faces_from_collection(Collection_ID, faces)
    print("deleted faces count: " + str(faces_count))

In [22]:
deleteFaces_collection(aws_accesskey,aws_secretaccess,myregion,"MyCollection",[<Face IDS>])

2 faces deleted:
ebe07fdb-a4ab-4b52-9235-dbd4800f4d81
ac5b2d79-a2d2-466c-aa66-eb7dc07b7828


2